# QuestionGeneration 评估问题生成与评估

以下代码将示范如何通过 llamaindex 生成一系列针对给定文本数据的“问题”。这些问题对于 FaithfulnessEvaluator 和 RelevancyEvaluator 评估工具有极大帮助

## 环境配置

本次实验的环境完全在本地执行，运行LLM的软件是 LM Studio，它能提供类似 OpenAI API 接口，这样方便其他人使用 OpenAI API 进行复现。具体配置：
- LLM 模型是 mistral-7b-instruct-v0.2.Q6_K
- Embeddeding 模型：BAAI/bge-base-en-v1.5

In [2]:
%pip install llama-index
%pip install pandas 
%pip install spacy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

In [4]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator, FaithfulnessEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    Response,
)
from llama_index.embeddings import resolve_embed_model
from llama_index.llms import OpenAI

## 加载本地模型

In [7]:
# bge-m3 embedding model
# https://huggingface.co/BAAI/bge-base-en-v1.5/tree/main
embed_model = resolve_embed_model("local:BAAI/bge-base-en-v1.5")

# Load LM Studio LLM model
llm = OpenAI(api_base="http://localhost:1234/v1", api_key="not-needed")

# Index the data
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, llm=llm,
)

## 加载本地数据

In [8]:
documents = SimpleDirectoryReader("data").load_data()

In [9]:
data_generator = DatasetGenerator.from_documents(documents, service_context=service_context)

d:\Applications\Miniconda3\envs\llamaindex-learning\Lib\site-packages\llama_index\evaluation\dataset_generation.py:184: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [10]:
eval_questions = data_generator.generate_questions_from_nodes()

INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTT

d:\Applications\Miniconda3\envs\llamaindex-learning\Lib\site-packages\llama_index\evaluation\dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [11]:
eval_questions

['In what year did Paul Graham first encounter a computer and start writing programs?',
 "What type of machine was Paul Graham's first experience with computing on?",
 'What programming language did Paul Graham use when he first started coding?',
 'Why was Paul Graham unable to write interesting programs on the IBM 1401?',
 'When did Paul Graham get his first microcomputer and what make/model was it?',
 "What did Paul's father use the word processor program for?",
 'Why did Paul switch from studying philosophy in college to Artificial Intelligence (AI)?',
 'What two things inspired Paul to pursue a career in AI during the mid 1980s?',
 'In what year did Paul attend college and start taking philosophy courses?',
 'How did Paul describe his experience with philosophy courses in college?',
 'In what year did Paul Graham first become interested in Artificial Intelligence (AI)?',
 "What two things specifically sparked Paul Graham's interest in AI?",
 'Which language did Paul Graham learn to

## 使用 llamaindex 的 FaithfulnessEvaluator 进行评估

In [12]:
# evaluator = RelevancyEvaluator(service_context=service_context)
faithfulness_evaluator = FaithfulnessEvaluator(service_context=service_context)

In [13]:
# 对源数据进行向量化
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

### 统计整体的召回成功率

In [ ]:
query_engine = vector_index.as_query_engine()
passing = 0

for i in range(10):
    print("第 " + str(i) + " 次查询")
    response_vector = query_engine.query(eval_questions[i])
    eval_result = faithfulness_evaluator.evaluate_response(
        query=eval_questions[i], response=response_vector
    )
    print(eval_result.passing)
    if eval_result.passing:
        passing += 1

# 输出召回成功率。此处抽样前10个问题，统计全部问题耗时过长
print(passing / 10)

#### 查看其中一个的召回结果

In [16]:
# 定义展示表格
# define jupyter display function
def display_eval_df(querys: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": querys,
            "Response": str(response),
            "Source": (
                response.source_nodes[0].node.get_content()[:1000] + "..."
            ),
            "Evaluation Result": eval_result.passing,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [17]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator.evaluate_response(
    query=eval_questions[1], response=response_vector
)

# 输出其中一个的结果
display_eval_df(eval_questions[1], response_vector, eval_result)

INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"


,Query,Response,Source,Evaluation Result
0,What type of machine was Paul Graham's first experience with computing on?,Paul Graham's first experience with computing was on an IBM System/360 mainframe at Harvard University in the late 1970s.,"What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep. The first programs I tried writing were on the IBM 1401 that our school district used for what was then called ""data processing."" This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights. The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the...",False
